<a href="https://colab.research.google.com/github/ekansh2004/MLProject/blob/main/Medical_Abv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit transformers torch pandas openpyxl sentence-transformers faiss-cpu
!pip install pyngrok
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
import anvil.server

anvil.server.connect("server_5RI2BHPKB7ZYRLZH3NTW5TY7-67ZYMSMSIU4WLEQT")

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
import textwrap
import faiss
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

device = torch.device('cpu')

checkpoint = "MBZUAI/LaMini-T5-738M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map=device,
    torch_dtype=torch.float32
)

index = None
document_store = []

def data_ingestion():
    global index, document_store
    csv_path = "/content/drive/My Drive/final_train.csv"

    df = pd.read_csv(csv_path)

    df['combined'] = df['LABEL'] + ' ' + df['ABV'].astype(str)

    text = df['combined'].to_string(index=False)

    with open("/content/temp.txt", "w") as file:
        file.write(text)

    with open("/content/temp.txt", "r") as file:
        document_text = file.read()

    text_chunks = textwrap.wrap(document_text, 500)

    documents = [chunk for chunk in text_chunks]

    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(documents)

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))

    document_store = documents

data_ingestion()

pipe = pipeline(
    'text2text-generation',
    model=base_model,
    tokenizer=tokenizer,
    max_length=256,
    do_sample=True,
    temperature=0.3,
    top_p=0.95
)

model = SentenceTransformer('all-MiniLM-L6-v2')

def retriever(query):
    global index, document_store
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, 5)
    return [document_store[i] for i in I[0]]

def qa(query):
    retrieved_docs = retriever(query)
    combined_docs = " ".join(retrieved_docs)
    input_text = f"Answer the question based on the following context:\n\n{combined_docs}\n\nQuestion: {query}\nAnswer:"
    response = pipe(input_text)
    return response[0]['generated_text']

@anvil.server.callable
def get_answer(query):
    return qa(query)

print("Anvil server is connected and ready to receive queries.")
anvil.server.wait_forever()

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Anvil server is connected and ready to receive queries.


KeyboardInterrupt: 

In [ ]:
csv_path = "/content/drive/My Drive/final_train.csv"
df = pd.read_csv(csv_path)
df.shape

(100000, 2)